In [7]:
%reload_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
from file_exchange import config
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
from gevent.event import AsyncResult
from gevent.pool import Group
from gevent.select import select
import gevent
from copy import deepcopy
import pandas as pd
import numpy as np
import random
import time
import os
import sys
import json

# File sharing

In [2]:
def get_users_pair(users):
    u1 = random.choice(users)
    u2 = random.choice(users)
    while u1 == u2:
        u2 = random.choice(users)
    return u1, u2

def save_conf(conf):
    with open(conf['output_dir']+"conf.json", "w") as f:
        conf2 = deepcopy(conf)
        conf2['period'] = str(conf2['period'])
        json.dump(conf2, f)
def load_conf(path):
    f = open(path+'conf.json', 'r')
    conf = json.load(f)
    conf['period'] = datetime.strptime(conf['period'], "%H:%M:%S") - \
                      datetime.strptime("00:00", "%H:%M")
    return conf

In [16]:
conf = config.default
conf['n_users'] = 10
conf['n_rounds'] = 30
conf['period'] = timedelta(seconds=3)
conf['output_dir'] = 'outputs/file_exchange_random_connected/_lalalal'
conf['send_strategy'] = 'random_connected'
print(conf)

address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)
    #users[i].update_state()

users_pairs = set()
while len(users_pairs) < conf['n_file_exchanges']:
    u1, u2 = get_users_pair(users)
    users_pairs.add((u1, u2))
    
for u in users:
    # Starts greenlet
    u.start()

gevent.sleep(2)

files_info=[]
for u1, u2 in users_pairs:
    try:
        file_info = u2.init_file_receive(u1)
        files_info.append(file_info)
    except:
        pass
n_file_exchanges = len(files_info)

gevent.joinall(users)

save_conf(conf)
print("Done")

{'minimum_node_availability': 0, 'n_users': 10, 'n_layers': 3, 'period': datetime.timedelta(0, 3), 'file_size': 11, 'gossip_size': 20, 'bandwidth': 10, 'output_dir': 'outputs/file_exchange_random_connected/_lalalal', 'n_file_chunks': 10, 'n_file_exchanges': 10, 'send_strategy': 'random_connected', 'header_size': 0.1, 'do_monitor': True, 'layer_threshold': 0.001, 'chunk_max_size': 2, 'n_rounds': 30, 'ack_size': 0}
Device 6jmefe6mxx3z (owned by Shawnda) will send file jxz5ja
Device skgeyze6whyk (owned by Jay) will send file 2kzz7o
Device jj9f24cv0efw (owned by Jack) will send file jio2jt
Device f6bcbabtw0kc (owned by Evelyne) will send file 94ou2a
Device 9b8d9yyztd60 (owned by Reita) will send file h79xwq
Device k2vuforhnf2w (owned by Reita) will send file ez5sgj
Device nticisrj6k2x (owned by Consuelo) will send file 8ym8y2
Device 4771icfdfm99 (owned by Tim) will send file fo4x3i
Device b85gw1i7et23 (owned by Reita) will send file fvnf7r
Device erz2ff91fjlw (owned by Jack) will send file

# Interpretation

In [15]:
path="outputs/file_exchange_random_connected/"
conf = load_conf(path)

def merge_results(conf):
    df = pd.DataFrame()
    for fn in os.listdir(conf['output_dir']):
        if not fn.endswith(".csv"):
            continue
            
        filepath = conf['output_dir']+fn
        try:
            # df=pd.read_csv(filepath, parse_dates=['Timestamp'],
            # index_col=['Timestamp'])
            df2 = pd.read_csv(filepath)
        except:
            print("Failed loading {}: {}".format(fn, sys.exc_info()))
            pass
        else:
            if(df.empty):
                df = df2
            else:
                df = df.append(df2,)
            print("Loaded dataframe {}.".format(fn))
            
    return df

merge_results(conf)
        

Loaded dataframe _lalalalz7f20v3sn4st.csv.
Loaded dataframe _lalalalck08dyqu2nzd.csv.
Loaded dataframe kd0musfhxbl5.csv.
Loaded dataframe _lalalalgtj7iusjg1qp.csv.
Loaded dataframe p68rtpv0knne.csv.
Loaded dataframe 5ibe831p46qm.csv.
Loaded dataframe jvy7ct69fofa.csv.
Loaded dataframe y6f6t7s0i5dx.csv.
Loaded dataframe netezq034ne7.csv.
Loaded dataframe a31pke2rcjzm.csv.
Loaded dataframe qyb2kkmh21pj.csv.
Loaded dataframe lcnx23mogrwo.csv.
Loaded dataframe _lalalalvcccb6rmv8zo.csv.
Loaded dataframe 0z7y4wlsxshg.csv.
Loaded dataframe a9giyh28ezav.csv.
Loaded dataframe 2puflgwso63s.csv.
Loaded dataframe iijx9b5w5xf5.csv.
Loaded dataframe w0luqezfya7k.csv.
Loaded dataframe 5mtn0qbd3cwu.csv.
Loaded dataframe of044pwfp9gk.csv.
Loaded dataframe 6sgm5hxgv0gg.csv.
Loaded dataframe q5c3c1qerkxc.csv.
Loaded dataframe ligy3blym01q.csv.
Loaded dataframe _lalalal37el8f9g2412.csv.
Loaded dataframe 7y2q83g04vj3.csv.
Loaded dataframe 7t76qc4e66lw.csv.
Loaded dataframe 2o5d1he21v2u.csv.
Loaded datafram

Loaded dataframe crs5rdyvvgso.csv.
Loaded dataframe evlky6hxu00r.csv.
Loaded dataframe _lalalaleq6pmvuypmes.csv.
Loaded dataframe e1sa9guqo8tx.csv.
Loaded dataframe _lalalalqzg6nfxhslc6.csv.
Loaded dataframe _lalalalw2knwxaeso23.csv.
Loaded dataframe _lalalalvt5cwuhgpbem.csv.
Loaded dataframe c4fr3eb86xjt.csv.
Loaded dataframe _lalalal4y540adg8han.csv.
Loaded dataframe mmcut6ecx6go.csv.
Loaded dataframe _lalalalljn7qw2c74yd.csv.
Loaded dataframe _lalalaln3obmps768o7.csv.
Loaded dataframe 92klvltmzxo6.csv.
Loaded dataframe _lalalalh3z45q59dp3l.csv.
Loaded dataframe veupnalbqqlm.csv.
Loaded dataframe zaz235sjl75h.csv.
Loaded dataframe bb2yh1e5lm0l.csv.
Loaded dataframe _lalalalx21xmb7d3aky.csv.
Loaded dataframe yjiok6vby4jn.csv.
Loaded dataframe fjs7tbrc4d7m.csv.
Loaded dataframe i65my5i802xo.csv.
Loaded dataframe bclx6dfe9rtr.csv.
Loaded dataframe n0nojwr06y1m.csv.
Loaded dataframe _lalalalxk8iuliik54z.csv.
Loaded dataframe sywb6w3tmg3r.csv.
Loaded dataframe _lalalalvnyu4oh4kv4i.csv.
Lo

,Unnamed: 0,t,addr,owner,file_id,chunk_id,type,sent,received,forwarded
0,0,85493.962168,z7f20v3sn4st,Norene,epmaek,1,MessType.CHUNK,0.0,2.1,0.0
1,1,85494.004751,z7f20v3sn4st,Norene,epmaek,1,MessType.ACK,0.1,0.0,0.0
2,2,85494.304659,z7f20v3sn4st,Norene,do8p5s,2,MessType.CHUNK,0.0,0.0,2.1
3,3,85494.571103,z7f20v3sn4st,Norene,epmaek,3,MessType.CHUNK,0.0,2.1,0.0
4,4,85494.594777,z7f20v3sn4st,Norene,epmaek,3,MessType.ACK,0.1,0.0,0.0
5,5,85494.695445,z7f20v3sn4st,Norene,epmaek,5,MessType.CHUNK,0.0,1.1,0.0
6,6,85494.718289,z7f20v3sn4st,Norene,epmaek,5,MessType.ACK,0.1,0.0,0.0
7,7,85494.757053,z7f20v3sn4st,Norene,5wou3f,1,MessType.CHUNK,0.0,0.0,2.1
8,8,85495.562798,z7f20v3sn4st,Norene,epmaek,0,MessType.CHUNK,0.0,2.1,0.0
9,9,85495.596606,z7f20v3sn4st,Norene,epmaek,0,MessType.ACK,0.1,0.0,0.0
